https://mr-nlp.github.io/posts/2022/07/mt-hf-pipeline/

In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install evaluate
!pip install rouge_score
!pip install sentence_transformers
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.1 MB/s eta 0:00

In [2]:
from datasets import load_dataset,get_dataset_split_names,load_from_disk
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
#from metrics import calcSBERTScore,loadModel
import os
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import pickle
from typing import List
import yaml
import time
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import pandas as pd
import numpy as np

nltk.download('punkt')
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
modelsbert_path = 'paraphrase-MiniLM-L6-v2'

In [3]:
dataset = load_dataset("wmt18",'cs-en','validation')

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/11046024 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3005 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2983 [00:00<?, ? examples/s]

Dataset wmt18 downloaded and prepared to /root/.cache/huggingface/datasets/wmt18/cs-en-data_dir=validation/1.0.0/555a687a5c9cfe8c9393a70305bfb58f26e8326f22dbf99767a28b298b6fa144. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

inputs = list(dataset['validation']['translation'][0:2])
inputs_cs = []
ground_truth_en = []
for inp in inputs:
  inputs_cs.append(str(inp['cs']))
  ground_truth_en.append(str(inp['en']))

outputs_en = []

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer.src_lang = "cs_CZ"

for input_text in inputs_cs:

  encoded_cs = tokenizer(input_text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_cs)
  output_en = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  outputs_en.append(output_en[0])

In [ ]:
results_dataframe_translation_wmt18 = pd.DataFrame()
results_dataframe_translation_wmt18['output_en'] = output_en
results_dataframe_translation_wmt18['ground_truth_en'] = ground_truth_en
results_dataframe_translation_wmt18.to_csv('results_dataframe_translation_wmt18.csv')

In [25]:
def loadModel(model_path):
  return SentenceTransformer(model_path)

def getSentences(text):
  return sent_tokenize(text)

def getEmbeddings(model, sentences):
  embeddings = model.encode(sentences)
  return embeddings

def getSimilarityMatrix(generated_embs, reference_embs):
  return cosine_similarity(generated_embs, reference_embs)

def getScore(sim_mat):
  
  # get maximum along reference text axis
  scores = np.max(sim_mat, axis = 0)

  # calculate mean
  final_score = np.mean(scores)

  return final_score

def calcSBERTScore(model, generated_texts, reference_texts, debug = False):

  score_final = 0

  # get sentences
  for i in range(len(generated_texts)):

    generated_text = generated_texts[i]
    reference_text = reference_texts[i]

    generated_sents = getSentences(generated_text)
    reference_sents = getSentences(reference_text)

    # get embeddings
    generated_embs = getEmbeddings(model, generated_sents)
    reference_embs = getEmbeddings(model, reference_sents)

    # calculate pairwise cosine similarity
    sim_mat = cosine_similarity(generated_embs, reference_embs)

    if debug:
      print("sim mat shape", sim_mat.shape)
      print(sim_mat)
    # get score
    score = getScore(sim_mat)
    score_final = score_final + score
  
  return score_final/len(generated_texts)

In [28]:
# Measure time for results_rouge
start_time = time.time()
rouge = evaluate.load('rouge')
results_rouge = rouge.compute(predictions=outputs_en, references=ground_truth_en)
end_time = time.time()
results_rouge_time = end_time - start_time

# Measure time for results_bertscore
start_time = time.time()
bertscore = evaluate.load("bertscore")
results_bertscore = bertscore.compute(predictions=outputs_en, references=ground_truth_en, lang="en")
end_time = time.time()
results_bertscore_time = end_time - start_time

# Measure time for results_sBERT
modelsbert = loadModel(modelsbert_path)
start_time = time.time()
results_sBERT = calcSBERTScore(modelsbert, outputs_en, ground_truth_en)
end_time = time.time()
results_sBERT_time = end_time - start_time

# Print the time taken for each prediction
print("Time taken for results_rouge: ", results_rouge_time, " seconds")
print("Time taken for results_bertscore: ", results_bertscore_time, " seconds")
print("Time taken for results_sBERT: ", results_sBERT_time, " seconds")

results = {
    'results_rouge':[results_rouge,results_rouge_time],
    'results_bertscore':[results_bertscore,results_bertscore_time],
    'results_sBERT':[results_sBERT,results_sBERT_time]
}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Time taken for results_rouge:  0.636526346206665  seconds
Time taken for results_bertscore:  9.349079847335815  seconds
Time taken for results_sBERT:  0.2031998634338379  seconds
